In [42]:
import nbimporter
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
import tensorflow as tf 
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import joblib
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.metrics import confusion_matrix
from utils import *
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm,metrics,preprocessing,tree
from sklearn.ensemble import GradientBoostingClassifier
from bs4 import BeautifulSoup
import cld3

In [28]:
wst_data=load_dataset('../data/chiphish/chspec_combine_western_full.json') 
data_wst=wst_data.iloc[:,:66] 
data_wst=data_wst.astype(float)
 
print(data_wst.shape)
 

(11204, 66)


In [29]:
eng_data=load_dataset('../data/engphish/chspec_combine_enphish_full.json') 
data_eng=eng_data.iloc[:,:66] 
data_eng=data_eng.astype(float)
print(data_eng.shape)

(15111, 66)


In [30]:
ch_data=load_dataset('../data/wstphish/chspec_combine_chphish_full.json') 
data_ch=ch_data.iloc[:,:66] 
data_ch=data_ch.astype(float)
print(data_ch.shape)

(1620, 66)


# Custom PWD exploiting LaSeTo predict wstphish testing data (refer to Sec 7.3)  

In [ ]:
# wst
train_ze,test_ze=train_test_split(data_wst,test_size=0.2,shuffle=True)
train_ze_x=train_ze.iloc[:,0:65]
train_ze_y=train_ze.iloc[:,65] 
test_ze_x=test_ze.iloc[:,0:65] 
test_ze_y=test_ze.iloc[:,65] 

scaler_entire_ze=preprocessing.MinMaxScaler()
scaler_entire_ze.fit(train_ze_x)
train_x_ze=scaler_entire_ze.transform(train_ze_x)
test_x_ze=scaler_entire_ze.transform(test_ze_x)
model_rf1_ze=RandomForestClassifier(criterion='gini', max_depth=55, min_samples_split=3, n_estimators=200)
model_rf_com_ze=model_rf1_ze.fit(train_x_ze,train_ze_y)

#eng
train_eng,test_eng=train_test_split(data_eng,test_size=0.2,shuffle=True)
train_eng_x=train_eng.iloc[:,0:65]
train_eng_y=train_eng.iloc[:,65] 
test_eng_x=test_eng.iloc[:,0:65] 
test_eng_y=test_eng.iloc[:,65] 

scaler_entire_eng=preprocessing.MinMaxScaler()
scaler_entire_eng.fit(train_eng_x)
train_x_eng=scaler_entire_eng.transform(train_eng_x)
test_x_eng=scaler_entire_eng.transform(test_eng_x)

print('combine_rf_whole:')
rf_start=time.time()
model_rf1_eng=RandomForestClassifier(criterion='gini', max_depth=55, min_samples_split=3, n_estimators=200)
model_rf_com_eng=model_rf1_eng.fit(train_x_eng,train_eng_y)
#chiphish
train_ch,test_ch=train_test_split(data_ch,test_size=0.2,shuffle=True)
train_ch_x=train_ch.iloc[:,0:65]
train_ch_y=train_ch.iloc[:,65] 
test_ch_x=test_ch.iloc[:,0:65] 
test_ch_y=test_ch.iloc[:,65] 

scaler_entire_ch=preprocessing.MinMaxScaler()
scaler_entire_ch.fit(train_ch_x)
train_x_ch=scaler_entire_ch.transform(train_ch_x)
test_x_ch=scaler_entire_ch.transform(test_ch_x)

print('combine_rf_whole:')
rf_start=time.time()
model_rf1_ch=RandomForestClassifier(criterion='gini', max_depth=55, min_samples_split=3, n_estimators=200)
model_rf_com_ch=model_rf1_ch.fit(train_x_ch,train_ch_y)


In [ ]:
# for wst testing data
tp=tn=fp=fn=0
te_ze_inds=test_ze.index 

for i in range(0,len(te_ze_inds)):
        ind=te_ze_inds[i]
        wst_info=wst_data.iloc[ind] 
        subfolder=wst_info['id']
        label=wst_info['label']
        ht_path="wst_dataset_11204/"+subfolder+"/page.html" 
        html=readHtmlFile(ht_path) 
        lan=laseto(html) 
        if 'zh' in lan: 
            dw=data_wst.iloc[ind][:-1] 
            data_before=pd.DataFrame(dw.values.reshape(1, -1),columns=train_ch_x.columns)                
            test_ch_da=scaler_entire_ch.transform(data_before)
            tp_ch,tn_ch,fp_ch,fn_ch=sep_metri(model_rf_com_ch, test_ch_da,label)
            tp=tp+tp_ch
            tn=tn+tn_ch
            fp=fp+fp_ch
            fn=fn+fn_ch
        elif 'en' in lan: 
            dw=data_wst.iloc[ind][:-1] 
            data_before=pd.DataFrame(dw.values.reshape(1, -1),columns=train_eng_x.columns) 
            test_en_da=scaler_entire_eng.transform(data_before) 
            tp_eng,tn_eng,fp_eng,fn_eng=sep_metri(model_rf_com_eng,test_en_da,label)
            tp=tp+tp_eng
            tn=tn+tn_eng
            fp=fp+fp_eng
            fn=fn+fn_eng
        else: 
            dw=data_wst.iloc[ind][:-1] 
            data_before=pd.DataFrame(dw.values.reshape(1, -1),columns=train_ze_x.columns)  
            test_ze_da=scaler_entire_ze.transform(data_before) 
            tp_ze,tn_ze,fp_ze,fn_ze=sep_metri(model_rf_com_ze,test_ze_da,label)
            tp=tp+tp_ze
            tn=tn+tn_ze
            fp=fp+fp_ze
            fn=fn+fn_ze
tpr=tp/(tp+fn)
tnr=tn/(tn+fp)
f1_score=2*(((tp/(tp+fp))*(tp/(tp+fn)))/((tp/(tp+fp))+(tp/(tp+fn))))
print('tpr is %f, tnr is %f, f1_score is %f'%(tpr,tnr,f1_score))
 
